In [1]:
import itertools
import numpy as np
from collections import defaultdict

In [2]:
with open('day19.txt') as f:
    lines = f.read().strip()
    scanners = lines.split('\n\n')

    scanner_list = []
    for scan in scanners:
        beacons = []
        for line in scan.split('\n'):
            line = line.strip()
            if line.startswith('--'):
                continue
            x, y, z = [int(v) for v in line.split(',')]
            beacons.append((x, y, z))
        scanner_list.append(beacons)

In [3]:
def adjust(point, d):
    ret = [point[0], point[1], point[2]]
    for i, p in enumerate(list(itertools.permutations([0, 1, 2]))):
        
        # Change x,y,z position -> [x,y,z], [y,x,z], [y,z,x] ...
        if d//8 == i:
            ret = [ret[p[0]], ret[p[1]], ret[p[2]]]

    if d % 2 == 1:
        ret[0] *= -1 # [x, -x]
    if (d//2) % 2 == 1:
        ret[1] *= -1 # [y, -y]
    if (d//4) % 2 == 1:
        ret[2] *= -1 # [z, -z]
    return ret

In [4]:
def man_dist(p1, p2):
    return np.abs(np.array(p1) - np.array(p2)).sum()

In [5]:
FINAL = set(scanner_list[0])

num_scanners = len(scanner_list)

shifts = [None for _ in range(num_scanners)]
shifts[0] = (0, 0, 0)

In [6]:
g_scanners = set([0])
b_scanners = set([x for x in range(1, num_scanners)])

In [7]:
scanner_adj = {}
for i in range(num_scanners):
    for d in range(48):
        scanner_adj[(i, d)] = [adjust(p, d) for p in scanner_list[i]]

In [8]:
len(scanner_adj) # = num. of scanners * 48 possible directions

1248

In [9]:
# Part 1
while b_scanners:
    found = None
    for b in b_scanners:
        if found:
            continue

        g_scan = [tuple([p[0], p[1], p[2]]) for p in FINAL]
        g_set = set(g_scan) # List of beacons found relative to scanner 0

        for b_dir in range(48):
            b_scan = scanner_adj[(b, b_dir)]
            shift_list = defaultdict(int)
            for bi in range(len(scanner_list[b])):
                for gi in range(len(g_scan)):
                    dx = -b_scan[bi][0] + g_scan[gi][0]
                    dy = -b_scan[bi][1] + g_scan[gi][1]
                    dz = -b_scan[bi][2] + g_scan[gi][2]
                    shift_list[(dx, dy, dz)] += 1
            for (dx, dy, dz), val in shift_list.items():
                if val >= 12:
                    shifts[b] = (dx, dy, dz) # Scanner location found relative to scanner 0 at (0,0,0)

                    print(f'FOUND scanner #{b} at coordinates: ({dx}, {dy}, {dz})')

                    for p in b_scan:
                        FINAL.add(tuple([p[0] + dx, p[1]+dy, p[2]+dz])) # Add new beacons to FINAL list of beacons
                    found = b

    assert found
    b_scanners.remove(found)
    g_scanners.add(found)

FOUND scanner #6 at coordinates: (8, 2, 1200)
FOUND scanner #7 at coordinates: (1268, 109, 1120)
FOUND scanner #10 at coordinates: (1248, 1282, 1274)
FOUND scanner #11 at coordinates: (-1177, -38, 1141)
FOUND scanner #4 at coordinates: (-2357, -31, 1149)
FOUND scanner #2 at coordinates: (-2411, -1258, 1286)
FOUND scanner #3 at coordinates: (-2467, 1174, 1308)
FOUND scanner #5 at coordinates: (-2413, 1277, 8)
FOUND scanner #1 at coordinates: (-3601, 1267, 49)
FOUND scanner #8 at coordinates: (-1121, 1233, -69)
FOUND scanner #9 at coordinates: (-1115, -1100, 1205)
FOUND scanner #14 at coordinates: (-2493, 1196, -1281)
FOUND scanner #15 at coordinates: (-3508, 1142, -1152)
FOUND scanner #16 at coordinates: (-3679, 2498, 42)
FOUND scanner #17 at coordinates: (-3659, 1211, 1246)
FOUND scanner #18 at coordinates: (-2370, 1185, -2367)
FOUND scanner #13 at coordinates: (-2336, 1296, -3507)
FOUND scanner #19 at coordinates: (-3509, 7, -1141)
FOUND scanner #20 at coordinates: (-1108, 1180, 1253)

In [10]:
print(f'Part 1: {len(FINAL)}')

Part 1: 313


In [11]:
# Part 2
distances = {}
for p1, p2 in itertools.combinations(shifts, 2):
    distances[(p1, p2)] = (man_dist(p1,p2))

max_key = max(distances, key = distances.get)

In [12]:
print(f'Part 2: {distances[max_key]} between {max_key[0]} and {max_key[1]}')

Part 2: 10656 between (-2336, 1296, -3507) and (1120, -1113, 1284)
